# 1. loading datasets

Training has to be done with dropout as data is not always present

In [43]:
import torch
import pandas as pd
import numpy as np
import time
import joblib
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F




In [44]:

from torch import nn 

# Check PyTorch version
print("PyTorch version:", torch.__version__)

# Check if CUDA is available
print("CUDA available:", torch.cuda.is_available())

# Get current CUDA device index (if available)
if torch.cuda.is_available():
    print("Current CUDA device index:", torch.cuda.current_device())
    print("CUDA device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("No CUDA devices found.")
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


PyTorch version: 2.5.0+cu118
CUDA available: True
Current CUDA device index: 0
CUDA device name: NVIDIA GeForce RTX 4070 SUPER


In [45]:
# Update with the correct path
# file_path = './Data/12_Sep_Readings/combined_data.csv'  
# file_path = './Data/15_Nov_Readings/augmented_rssi_dataset.csv'
file_path = './Data/25-02-10/cleaned_df.csv'
model_path = './models/CNN'
cleaned_df = pd.read_csv(file_path)

In [46]:
cleaned_df

Tx_0 RSSI  Tx_1 RSSI  Tx_2 RSSI  Tx_3 RSSI  Tx_4 RSSI  Tx_5 RSSI  \
0           -61        -74        -79        127        -81        -78   
1           -61        -73        -78        -71        127        -78   
2           -62        -75        -78        -72        -79        127   
3           -61        -73        -80        -71        -83        -78   
4           -61        -75        -78        -71        -84        -78   
...         ...        ...        ...        ...        ...        ...   
6395        -79        -69        -81        -74        -70        -61   
6396        -77        -69        -77        -71        -70        -61   
6397        -77        -69        -81        -73        -70        -61   
6398        -80        -69        -77        -72        -71        -61   
6399        -77        -68        -81        -72        -70        -61   

      Tx_6 RSSI  Tx_7 RSSI  X_Coord  Y_Coord  
0           127        -84        0        0  
1           127        127        0        0  
2           -84        127        0        0  
3           127        127        0        0  
4           -84        -86        0        0  
...         ...        ...      ...      ...  
6395        -78        -68        6        0  
6396        -79        -67        6        0  
6397        -79        -66        6        0  
6398        -76        -67        6        0  
6399        -70        -68        6        0  

[6400 rows x 10 columns]

In [47]:
from common_utils import preprocess_dataset

# X = cleaned_df[['Tx_0 RSSI', 'Tx_1 RSSI', 'Tx_2 RSSI', 'Tx_3 RSSI', 'Tx_4 RSSI', 'Tx_5 RSSI', 'Tx_6 RSSI', 'Tx_7 RSSI']]
# X = cleaned_df[['Tx_0 RSSI_Avg', 'Tx_1 RSSI_Avg', 'Tx_2 RSSI_Avg', 'Tx_3 RSSI_Avg', 'Tx_4 RSSI_Avg', 'Tx_5 RSSI_Avg', 'Tx_6 RSSI_Avg', 'Tx_7 RSSI_Avg']]
X = cleaned_df[['Tx_0 RSSI', 'Tx_1 RSSI', 'Tx_2 RSSI', 'Tx_3 RSSI', 'Tx_4 RSSI', 'Tx_5 RSSI', 'Tx_6 RSSI', 'Tx_7 RSSI']]
Y = cleaned_df[['X_Coord', 'Y_Coord']]



In [48]:
grid_data = {}

# Collect RSSI values per (X_Coord, Y_Coord)
for index, row in cleaned_df.iterrows():
    x, y = int(row['X_Coord']), int(row['Y_Coord'])
    
    # Initialize empty list for this location if not already present
    if (x, y) not in grid_data:
        grid_data[(x, y)] = []
    
    # Append RSSI values (Tx_0 to Tx_7)
    grid_data[(x, y)].append(row[X.columns].values)

# Define parameters
num_tx = 8      # Number of transmitters
seq_length = 8  # Sequence length (8 samples per training instance)
stride = 1      # Sliding window step (adjustable)
max_sequences = 50  # Maximum sequences per grid cell to limit dataset size

# Create a list to store all sequences
training_samples = []

# Generate sequences for CNN training
for (x, y), rssi_values in grid_data.items():
    rssi_array = np.array(rssi_values)  # Convert to NumPy array
    
    if rssi_array.shape[0] < seq_length:
        continue  # Skip locations with fewer than 8 samples

    # Extract multiple 8-sample sequences using a sliding window
    num_samples = rssi_array.shape[0]

    for start in range(0, num_samples - seq_length + 1, stride):
        # Get an 8-sample window
        window = rssi_array[start:start + seq_length].T  # Shape: (Tx, 8 samples)

        # Store sample with label (x, y)
        training_samples.append((window, (x, y)))

In [49]:
if training_samples:
    X_Sequence = np.array([sample[0] for sample in training_samples])  # Shape: (num_samples, Tx, 8)
    y_Sequence = np.array([sample[1] for sample in training_samples])  # Shape: (num_samples, 2) -> (X_Coord, Y_Coord)
else:
    X_Sequence = np.array([])
    y_Sequence = np.array([])

print("Generated training data shape:", X_Sequence.shape)
print("Generated labels shape:", y_Sequence.shape)

Generated training data shape: (5952, 8, 8)
Generated labels shape: (5952, 2)


In [50]:
for i in range(2):
    print(f"Sample {i+1}:")
    print(X_Sequence[i])  # Prints the 8x8 RSSI matrix
    print(f"Label: {y_Sequence[i]}")  # Prints (X_Coord, Y_Coord)
    print("-" * 40)

Sample 1:
[[-61 -61 -62 -61 -61 -61 -61 -61]
 [-74 -73 -75 -73 -75 -75 -75 -75]
 [-79 -78 -78 -80 -78 -78 -78 -79]
 [127 -71 -72 -71 -71 -71 -71 -71]
 [-81 127 -79 -83 -84 -82 -80 -82]
 [-78 -78 127 -78 -78 -80 -78 -78]
 [127 127 -84 127 -84 -82 -84 127]
 [-84 127 127 127 -86 -85 -84 127]]
Label: [0 0]
----------------------------------------
Sample 2:
[[-61 -62 -61 -61 -61 -61 -61 -61]
 [-73 -75 -73 -75 -75 -75 -75 -75]
 [-78 -78 -80 -78 -78 -78 -79 -79]
 [-71 -72 -71 -71 -71 -71 -71 -71]
 [127 -79 -83 -84 -82 -80 -82 -81]
 [-78 127 -78 -78 -80 -78 -78 -78]
 [127 -84 127 -84 -82 -84 127 127]
 [127 127 127 -86 -85 -84 127 127]]
Label: [0 0]
----------------------------------------


In [51]:
from sklearn.model_selection import train_test_split

# Perform train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X_Sequence, y_Sequence, test_size=0.2, random_state=42
)

import torch
import numpy as np

# Convert (X_Coord, Y_Coord) to a single class index
y_train = np.array([x * 8 + y for x, y in y_train])
y_test = np.array([x * 8 + y for x, y in y_test])

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)  # Must be long for classification
y_test = torch.tensor(y_test, dtype=torch.long)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train, X_test, y_train, y_test = X_train.to(device), X_test.to(device), y_train.to(device), y_test.to(device)


# 2. NN model

In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Convolutional Layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)   # output size = 8x8x32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)  # output size = 4x4x64
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # output size = 4x4x128
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)  # output size = 4x4x256

        # Fully Connected Layers
        self.fc1 = nn.Linear(256 * 4 * 4, 512)  # 4x4x256 -> 512
        self.fc2 = nn.Linear(512, 256)  # 512-> 256
        self.fc3 = nn.Linear(256, 64)  # 256-> 64

        # Dropout for regularization
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.unsqueeze(1)  
        x = F.relu(self.conv1(x))  
        x = F.relu(self.conv2(x))  
        x = F.relu(self.conv3(x)) 
        x = F.relu(self.conv4(x)) 

        x = x.view(x.size(0), -1)  # Flatten

        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)  # Output 64 logits (for 64 locations)

        return x

In [53]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [54]:
print(model)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=64, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [55]:
# Training loop
num_epochs = 200
patience = 40
best_val_loss = float('inf')
epochs_without_improvement = 0

In [56]:
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train)  # Shape (batch_size, 64)
    loss = criterion(outputs, y_train)  # `y_train` is shape (batch_size,)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_test)
        val_loss = criterion(val_outputs, y_test)

    train_losses.append(loss.item())
    val_losses.append(val_loss.item())

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print(f"Early stopping at epoch {epoch + 1}")
        break

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

Epoch [10/200], Train Loss: 2.8745, Val Loss: 2.2965
Epoch [20/200], Train Loss: 1.7982, Val Loss: 1.4281
Epoch [30/200], Train Loss: 1.2498, Val Loss: 1.0556
Epoch [40/200], Train Loss: 0.8841, Val Loss: 0.8652
Epoch [50/200], Train Loss: 0.6464, Val Loss: 0.7668
Epoch [60/200], Train Loss: 0.4582, Val Loss: 0.6521
Epoch [70/200], Train Loss: 0.2834, Val Loss: 0.5952
Epoch [80/200], Train Loss: 0.1809, Val Loss: 0.5501
Epoch [90/200], Train Loss: 0.1188, Val Loss: 0.5274
Epoch [100/200], Train Loss: 0.0874, Val Loss: 0.5263
Epoch [110/200], Train Loss: 0.0761, Val Loss: 0.5573
Epoch [120/200], Train Loss: 0.0529, Val Loss: 0.5113
Epoch [130/200], Train Loss: 0.0419, Val Loss: 0.5598
Early stopping at epoch 138


In [57]:
# Set model to evaluation mode
model.eval()

# Choose a sample input from the test set
# Assuming you have already split your data into X_test (input) and y_test (targets)
inputs, targets = X_test, y_test  # Replace X_test, y_test with your actual test set

# Select only the first 10 samples from the batch
inputs = inputs[:10]
actual_targets = targets[:10]  # Corresponding actual coordinates for the test set

# Reshape inputs to fit the model (if needed)
# inputs = inputs.unsqueeze(1)  # Add channel dimension, shape: (10, 1, 8, 20)

# Perform the forward pass
with torch.no_grad():
    # Forward pass to get the model's predictions
    preds = model(inputs)  # Output shape: (10, 2) for X_Coord and Y_Coord predictions

predicted_coords = torch.round(preds).to(torch.int)
# Display predicted coordinates (X, Y) and actual targets
print("Predicted X and Y coordinates:")
print(predicted_coords.numpy())  

print("\nActual X and Y coordinates:")
print(actual_targets.numpy()) 


Predicted X and Y coordinates:


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
plt.plot(range(len(train_losses)), train_losses, label='Train Loss')
plt.plot(range(len(val_losses)), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Losses')
plt.show()